# iNaturalist status updates by state - QLD

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv` (statuses joined to taxa names), generate lists to update iNaturalist statuses

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the state conservation and sensitive lists
3. Prep fields incl IUCN equivalent mappings and matching to iNat taxonomy  
4. Merge and compare the state and inaturalist lists
5. Create update/removals list
6. Create additions list
7. Save files

## 1. Read in the inaturalist statuses & filter out QLD


# iNaturalist statuses

In [1]:
import pandas as pd
import sys
import os
projectdir = os.path.dirname(os.getcwd()) + "/" # parent dir of cwd
sourcedir = projectdir + "data/in/"
sys.path.append(os.path.abspath(projectdir + "notebooks/includes/"))
import list_functions  as lf

# read in the statuses file
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str)

# filter out ACT entries
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])

inatstatuses = filter_state_statuses("Qld|QLD|Queensland|QUEENSLAND|QL", ".qld.")
inatstatuses = inatstatuses.add_prefix("inat_")
inatstatuses.groupby(['inat_status']).size()


inat_status
Confidential              35
Critically Endangered     96
EN                         7
Endangered               230
Extinct                    3
Extinct in the wild       17
LC                        20
Least Concern              1
Least concern             44
NE                         4
NT                         5
Near Threatened          212
Near threatened            2
Not evaluated              1
SL                         1
Significant                1
Special least concern    781
VU                        10
Vulnerable               464
endangered                35
dtype: int64

## 2. Read in the State lists
Read in the state lists 
 * Set conservation list value `authority` = "Queensland Nature Conservation Act 1992"
 * Set sensitive list value: `geoprivacy` = `obscured`


In [2]:
%%script echo skipping # comment this line to download dataset from lists.ala.org.au the web and save locally
# Download lists data. Retrieve binomial and trinomial names from GBIF. Save locally to CSV

sensitivelist = lf.download_ala_specieslist("https://lists.ala.org.au/ws/speciesListItems/dr493?max=10000&includeKVP=true") ### TO DO: THIS IS dr493 in PROD
sensitivelist = lf.kvp_to_columns(sensitivelist)
sensitivelist.to_csv(sourcedir + "/state-lists/qld-ala-sensitive.csv", index=False)

conservationlist = lf.download_ala_specieslist("https://lists.ala.org.au/ws/speciesListItems/dr652?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist.to_csv(sourcedir + "/state-lists/qld-ala-conservation.csv", index=False)


skipping # comment this line to download dataset from lists.ala.org.au the web and save locally


In [3]:
# Read lists and merge
sensitivelist = pd.read_csv(sourcedir + "/state-lists/qld-ala-sensitive.csv", dtype=str)
conservationlist = pd.read_csv(sourcedir + "/state-lists/qld-ala-conservation.csv", dtype=str)
sensitivelist['geoprivacy'] = 'obscured'
sensitivelist['authority'] = "Qld Department of Environment and Science"
conservationlist['authority'] = "Queensland Nature Conservation Act 1992"
conservationlist = conservationlist.rename(columns={'Wild Net Taxon ID': 'WildNetTaxonID'})
statelist = conservationlist[['name','WildNetTaxonID','status','authority']].merge(sensitivelist[['name','WildNetTaxonID','geoprivacy','status','authority']], how="outer",on='WildNetTaxonID',suffixes=('_conservation', '_sensitive'))
statelist['scientificName'] = statelist.apply(lambda x: x['name_conservation'] if pd.isnull(x['name_sensitive']) else x['name_sensitive'], axis=1)
statelist['status'] = statelist.apply(lambda x: x['status_conservation'] if pd.isnull(x['status_sensitive']) else x['status_sensitive'], axis=1)
statelist['authority'] = statelist.apply(lambda x: x['authority_conservation'] if pd.isnull(x['authority_sensitive']) else x['authority_sensitive'], axis=1)
statelist = statelist.add_prefix("state_")
print("Conservation list entries:" + str(len(conservationlist)))
print("Sensitive list entries:" + str(len(sensitivelist)))
statelist

Conservation list entries:1351
Sensitive list entries:960


,state_name_conservation,state_WildNetTaxonID,state_status_conservation,state_authority_conservation,state_name_sensitive,state_geoprivacy,state_status_sensitive,state_authority_sensitive,state_scientificName,state_status,state_authority
0,Adelotus brevis,706,Vulnerable,Queensland Nature Conservation Act 1992,NaN,NaN,NaN,NaN,Adelotus brevis,Vulnerable,Queensland Nature Conservation Act 1992
1,Philoria knowlesi,36765,Endangered,Queensland Nature Conservation Act 1992,Philoria knowlesi,obscured,Endangered,Qld Department of Environment and Science,Philoria knowlesi,Endangered,Qld Department of Environment and Science
2,Philoria kundagungan,687,Endangered,Queensland Nature Conservation Act 1992,Philoria kundagungan,obscured,Endangered,Qld Department of Environment and Science,Philoria kundagungan,Endangered,Qld Department of Environment and Science
3,Assa darlingtoni,693,Vulnerable,Queensland Nature Conservation Act 1992,NaN,NaN,NaN,NaN,Assa darlingtoni,Vulnerable,Queensland Nature Conservation Act 1992
4,Crinia tinnula,686,Vulnerable,Queensland Nature Conservation Act 1992,NaN,NaN,NaN,NaN,Crinia tinnula,Vulnerable,Queensland Nature Conservation Act 1992
...,...,...,...,...,...,...,...,...,...,...,...
1987,NaN,34778,NaN,NaN,Phlegmariurus phlegmaria,obscured,Least concern,Qld Department of Environment and Science,Phlegmariurus phlegmaria,Least concern,Qld Department of Environment and Science
1988,NaN,40557,NaN,NaN,Phlegmariurus proliferus,obscured,NaN,Qld Department of Environment and Science,Phlegmariurus proliferus,NaN,Qld Department of Environment and Science
1989,NaN,12057,NaN,NaN,Lindsaea pulchella var. blanda,obscured,NaN,Qld Department of Environment and Science,Lindsaea pulchella var. blanda,NaN,Qld Department of Environment and Science
1990,NaN,39465,NaN,NaN,Tectaria devexa,obscured,Least concern,Qld Department of Environment and Science,Tectaria devexa,Least concern,Qld Department of Environment and Science


### 4. Equivalent IUCN statuses

In [4]:
# iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild','Extinct'}
statelist.groupby(['state_status_conservation','state_status_sensitive'],dropna=False).count()

state_name_conservation  \
state_status_conservation state_status_sensitive                            
Critically Endangered     Critically Endangered                        35   
                          NaN                                         111   
Endangered                Endangered                                   82   
                          NaN                                         230   
Extinct                   NaN                                           6   
Extinct in the wild       Extinct in the wild                           7   
                          NaN                                          19   
Near Threatened           Near Threatened                              61   
                          NaN                                         219   
Vulnerable                Vulnerable                                  134   
                          NaN                                         447   
NaN                       Least concern                                 0   
                          Near Threatened                               0   
                          Special least concern                         0   
                          NaN                                           0   

                                                  state_WildNetTaxonID  \
state_status_conservation state_status_sensitive                         
Critically Endangered     Critically Endangered                     35   
                          NaN                                      111   
Endangered                Endangered                                82   
                          NaN                                      230   
Extinct                   NaN                                        6   
Extinct in the wild       Extinct in the wild                        7   
                          NaN                                       19   
Near Threatened           Near Threatened                           61   
                          NaN                                      219   
Vulnerable                Vulnerable                               134   
                          NaN                                      447   
NaN                       Least concern                            147   
                          Near Threatened                            1   
                          Special least concern                    363   
                          NaN                                      130   

                                                  state_authority_conservation  \
state_status_conservation state_status_sensitive                                 
Critically Endangered     Critically Endangered                             35   
                          NaN                                              111   
Endangered                Endangered                                        82   
                          NaN                                              230   
Extinct                   NaN                                                6   
Extinct in the wild       Extinct in the wild                                7   
                          NaN                                               19   
Near Threatened           Near Threatened                                   61   
                          NaN                                              219   
Vulnerable                Vulnerable                                       134   
                          NaN                                              447   
NaN                       Least concern                                      0   
                          Near Threatened                                    0   
                          Special least concern                              0   
                          NaN                                                0   

                                                  state_name_sensitive  \
state_status

In [5]:
#(50:CE, 30:V, 40:E, 20:NT, EX:70)
iucnStatusMappings = {
    'Least concern':'10',
    'Special least concern':'10',
    'Critically Endangered':'50',
    'Endangered':'40',
    'Vulnerable':'30',
    'Extinct':'70',
    'Extinct in the wild':'70',
    'Near Threatened':'20'
}

### 5. Determine best place ID to use

In [6]:
inatstatuses.groupby(['inat_place_id','inat_place_name','inat_place_display_name'])['inat_place_id'].count()
# looks like 7308

inat_place_id  inat_place_name        inat_place_display_name      
                                                                          5
144315         Brisbane City          Brisbane City                       2
153119         South East Queensland  South East Queensland, QL, AU       1
18870          Cairns - Pt B          Cairns - Pt B, QL, AU               1
19232          Yarrabah               Yarrabah, QL, AU                    1
6744           Australia              Australia                          13
7308           Queensland             Queensland, AU                   1946
Name: inat_place_id, dtype: int64

In [16]:
# set placeid
place_id = 7308
# get the inaturalist taxonomy matches for additions 
inattaxa = pd.read_csv(sourcedir + "inaturalist-australia-9/inaturalist-australia-9-taxa.csv",dtype=str,usecols=['id','name','rank','observations_count','is_active'])
inattaxa = inattaxa[inattaxa['is_active'] == 't']
inattaxa = inattaxa.rename(columns = {'id':'taxon_id','name':'taxon_name'})
inattaxa = inattaxa.add_prefix("inat_")
statelist = statelist[['state_scientificName','state_status','state_geoprivacy', 'state_WildNetTaxonID','state_authority']].merge(inattaxa,how="left",left_on='state_scientificName',right_on='inat_taxon_name',suffixes=(None,'_inat'))
statelist

,state_scientificName,state_status,state_geoprivacy,state_WildNetTaxonID,state_authority,inat_taxon_id,inat_taxon_name,inat_rank,inat_observations_count,inat_is_active
0,Adelotus brevis,Vulnerable,NaN,706,Queensland Nature Conservation Act 1992,22084,Adelotus brevis,species,523,t
1,Philoria knowlesi,Endangered,obscured,36765,Qld Department of Environment and Science,1363722,Philoria knowlesi,species,0,t
2,Philoria kundagungan,Endangered,obscured,687,Qld Department of Environment and Science,22055,Philoria kundagungan,species,1,t
3,Assa darlingtoni,Vulnerable,NaN,693,Queensland Nature Conservation Act 1992,25240,Assa darlingtoni,species,38,t
4,Crinia tinnula,Vulnerable,NaN,686,Queensland Nature Conservation Act 1992,25234,Crinia tinnula,species,80,t
...,...,...,...,...,...,...,...,...,...,...
2027,Phlegmariurus phlegmaria,Least concern,obscured,34778,Qld Department of Environment and Science,559037,Phlegmariurus phlegmaria,species,226,t
2028,Phlegmariurus proliferus,NaN,obscured,40557,Qld Department of Environment and Science,956571,Phlegmariurus proliferus,species,2,t
2029,Lindsaea pulchella var. blanda,NaN,obscured,12057,Qld Department of Environment and Science,NaN,NaN,NaN,NaN,NaN
2030,Tectaria devexa,Least concern,obscured,39465,Qld Department of Environment and Science,778248,Tectaria devexa,species,537,t


In [17]:
statelist[statelist['state_scientificName'] == 'Diomedea antipodensis gibsoni']

,state_scientificName,state_status,state_geoprivacy,state_WildNetTaxonID,state_authority,inat_taxon_id,inat_taxon_name,inat_rank,inat_observations_count,inat_is_active
48,Diomedea antipodensis gibsoni,Vulnerable,NaN,27768,Queensland Nature Conservation Act 1992,NaN,NaN,NaN,NaN,NaN


In [8]:
mergedstatuses = statelist.merge(inatstatuses,how="outer",left_on='state_scientificName',right_on='inat_scientificName')

In [9]:

# add extra fields 
# add some extra fields
mergedstatuses['place_id'] = str(place_id)
mergedstatuses['username'] = 'peggydnew'
mergedstatuses['description'] = "Todo"
mergedstatuses['state_url'] = "https://apps.des.qld.gov.au/species-search/details/?id=" + mergedstatuses['state_WildNetTaxonID'].astype(str)
mergedstatuses['state_iucn_equivalent'] = mergedstatuses['state_status'].map(iucnStatusMappings).fillna('30') # map to dictionary, Vulnerable default
mergedstatuses['state_status'] = mergedstatuses['state_status'].fillna('Confidential')
mergedstatuses['state_geoprivacy'] = mergedstatuses['state_geoprivacy'].fillna('open')
mergedstatuses['inat_taxon_id'] = mergedstatuses['inat_taxon_id_y'].fillna(mergedstatuses['inat_taxon_id_x'])
mergedstatuses['inat_scientificName'] = mergedstatuses['inat_scientificName'].fillna(mergedstatuses['inat_taxon_name'])
confidentialmsg = "Listed as Confidential - refer to https://www.data.qld.gov.au/dataset/queensland-confidential-species"
mergedstatuses['description'] = mergedstatuses.apply(lambda x: confidentialmsg if x['state_geoprivacy'] == "obscured" else "", axis=1)

# UPDATE: inat status and state status both exist
# REMOVE: inat status exists, state status does not
# ADD: state status exists, inat status does not (matching taxon)
# NO MATCH: state status exists, inat taxa not found
mergedstatuses['action'] = 'na'
mergedstatuses.loc[mergedstatuses['inat_id'].notnull() & mergedstatuses['state_scientificName'].notnull(), 'action'] = "UPDATE"
mergedstatuses.loc[mergedstatuses['inat_id'].notnull() & mergedstatuses['state_scientificName'].isnull(), 'action'] = "REMOVE"
mergedstatuses.loc[mergedstatuses['inat_id'].isnull() & mergedstatuses['inat_taxon_id'].notnull(), 'action'] = "ADD"
mergedstatuses.loc[mergedstatuses['inat_id'].isnull() & mergedstatuses['inat_taxon_id'].isnull(), 'action'] = "NO MATCH"

# only update those with different values 
mergedstatuses['action'] = mergedstatuses.apply(lambda x: "NO CHANGE" if (x['action'] == "UPDATE") & ((x['state_status'] == x['inat_status']) & (x['state_geoprivacy'] == x['inat_geoprivacy'] ) & (x['state_geoprivacy'] == x['inat_geoprivacy']) & (x['state_iucn_equivalent'] == x['inat_iucn'])) else x['action'], axis=1)

# display
mergedstatusesprintfriendly = mergedstatuses[['action','inat_id','inat_taxon_id','state_scientificName','inat_scientificName', 'state_status','inat_status','state_geoprivacy','inat_geoprivacy','state_iucn_equivalent','inat_iucn','state_authority','inat_authority','state_url','inat_url','inat_description','inat_place_display_name','inat_current_synonymous_taxon_ids']]

#mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
#mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Confidential')
#mergedstatuses['new_description'] = "Listed as Confidential - refer to https://www.data.qld.gov.au/dataset/queensland-confidential-species"


In [10]:
mergedstatusesprintfriendly

,action,inat_id,inat_taxon_id,state_scientificName,inat_scientificName,state_status,inat_status,state_geoprivacy,inat_geoprivacy,state_iucn_equivalent,inat_iucn,state_authority,inat_authority,state_url,inat_url,inat_description,inat_place_display_name,inat_current_synonymous_taxon_ids
0,NO CHANGE,267024,22084,Adelotus brevis,Adelotus brevis,Vulnerable,Vulnerable,open,open,30,30,Queensland Nature Conservation Act 1992,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,https://apps.des.qld.gov.au/species-search/det...,vulnerable,"Queensland, AU",
1,ADD,NaN,1363722,Philoria knowlesi,Philoria knowlesi,Endangered,NaN,obscured,NaN,40,NaN,Qld Department of Environment and Science,NaN,https://apps.des.qld.gov.au/species-search/det...,NaN,NaN,NaN,NaN
2,UPDATE,152550,22055,Philoria kundagungan,Philoria kundagungan,Endangered,Endangered,obscured,obscured,40,30,Qld Department of Environment and Science,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,https://apps.des.qld.gov.au/species-search/det...,,"Queensland, AU",
3,ADD,NaN,25240,Assa darlingtoni,Assa darlingtoni,Vulnerable,NaN,open,NaN,30,NaN,Queensland Nature Conservation Act 1992,NaN,https://apps.des.qld.gov.au/species-search/det...,NaN,NaN,NaN,NaN
4,NO CHANGE,271456,25234,Crinia tinnula,Crinia tinnula,Vulnerable,Vulnerable,open,open,30,30,Queensland Nature Conservation Act 1992,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,https://apps.des.qld.gov.au/species-search/det...,,"Queensland, AU",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2654,REMOVE,271792,984977,NaN,Lastreopsis tripinnata,Confidential,Special least concern,open,open,30,30,NaN,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,https://apps.des.qld.gov.au/species-search/det...,,"Queensland, AU",
2655,REMOVE,271794,986296,NaN,Lastreopsis wurunuran,Confidential,Special least concern,open,open,30,30,NaN,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,https://apps.des.qld.gov.au/species-search/det...,,"Queensland, AU",
2656,REMOVE,272007,986405,NaN,Parapolystichum windsorense,Confidential,Special least concern,open,open,30,30,NaN,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,https://apps.des.qld.gov.au/species-search/det...,,"Queensland, AU",
2657,REMOVE,272057,987744,NaN,Platycerium hillii,Confidential,Special least concern,open,open,30,30,NaN,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,https://apps.des.qld.gov.au/species-search/det...,,"Queensland, AU",


In [11]:
mergedstatuses.groupby('action').size()

action
ADD          186
NO CHANGE    451
NO MATCH     504
REMOVE       641
UPDATE       877
dtype: int64

## Updates
Updates match cleanly to an iNaturalist taxon and existing status. We'll only do an update if the status, geoprivacy or iucn values are different.

In [12]:
# Headers:
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['action'].isin(['UPDATE','REMOVE'])])
updates = updates[['action','state_scientificName','inat_id','inat_taxon_id','state_status','state_iucn_equivalent','state_authority','state_url','state_geoprivacy','place_id','username','description']]
updates.columns = updates.columns.str.replace("state_", "", regex=True)
updates.columns = updates.columns.str.replace("inat_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
2,UPDATE,Philoria kundagungan,152550,22055,Endangered,40,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
5,UPDATE,Mixophyes fleayi,152612,22064,Endangered,40,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
8,UPDATE,Rheobatrachus silus,272167,26674,Extinct in the wild,70,Queensland Nature Conservation Act 1992,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
9,UPDATE,Rheobatrachus vitellinus,272168,26672,Extinct,70,Queensland Nature Conservation Act 1992,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
10,UPDATE,Taudactylus acutirostris,272311,25294,Extinct in the wild,70,Queensland Nature Conservation Act 1992,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
...,...,...,...,...,...,...,...,...,...,...,...,...
2654,REMOVE,NaN,271792,984977,Confidential,30,NaN,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
2655,REMOVE,NaN,271794,986296,Confidential,30,NaN,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
2656,REMOVE,NaN,272007,986405,Confidential,30,NaN,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
2657,REMOVE,NaN,272057,987744,Confidential,30,NaN,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,


Notes on these, which should all be manually handled:
https://www.inaturalist.org/taxa/1357800 _Eulophia pulchra_ search on ALA returns _Oeceoclades pelorica_ (or Eulophia pelorica) Vulnerable in Qld
https://www.inaturalist.org/taxa/369238 _Dendrobium mortii_ is known in Qld as _Dockrillia mortii_ with a status of SLC.
https://www.inaturalist.org/taxa/631112 _Trichomanes endlicherianum_ is known as _Polyphlebium endlicherianum_ and on the list with a status of Vulnerable in Qld
... etc

## Additions
Match to an inat taxon and have a new sensitive or conservation status

In [13]:
# Headers:
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id

additions = pd.DataFrame(mergedstatuses[mergedstatuses['action'] == "ADD"])
additions = additions[['action','state_scientificName','inat_id','inat_taxon_id','state_status','state_iucn_equivalent','state_authority','state_url','state_geoprivacy','place_id','username','description']]
additions = additions.rename(columns={'state_scientificName':'Taxon_Name',
                                      'state_status':'Status',
                                      'state_authority':'Authority',
                                      'state_iucn_equivalent':'IUCN_equivalent',
                                      'description':'Description',
                                      'place_id':'iNaturalst_Place_ID',
                                      'state_url':'url',
                                      'state_geoprivacy':'taxon_Geoprivacy',
                                      'inat_taxon_id':'taxon_id'})
additions

,action,Taxon_Name,inat_id,taxon_id,Status,IUCN_equivalent,Authority,url,taxon_Geoprivacy,iNaturalst_Place_ID,username,Description
1,ADD,Philoria knowlesi,NaN,1363722,Endangered,40,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
3,ADD,Assa darlingtoni,NaN,25240,Vulnerable,30,Queensland Nature Conservation Act 1992,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
48,ADD,Diomedea antipodensis antipodensis,NaN,1506785,Vulnerable,30,Queensland Nature Conservation Act 1992,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
53,ADD,Thalassarche carteri,NaN,113965,Vulnerable,30,Queensland Nature Conservation Act 1992,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
81,ADD,Cacatua leadbeateri,NaN,116835,Endangered,40,Queensland Nature Conservation Act 1992,https://apps.des.qld.gov.au/species-search/det...,open,7308,peggydnew,
...,...,...,...,...,...,...,...,...,...,...,...,...
1988,ADD,Macrozamia,NaN,129824,Confidential,30,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
2002,ADD,Hydnophytum,NaN,425697,Confidential,30,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
2004,ADD,Myrmecodia,NaN,347765,Confidential,30,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
2009,ADD,Huperzia,NaN,49144,Confidential,30,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....


In [14]:
# write these to output files
mergedstatusesprintfriendly.to_csv(projectdir + "data/out/summaries/qld.csv",index=False)
updates.to_csv(projectdir + "data/out/updates-qld.csv", index=False)
additions.to_csv(projectdir + "data/out/additions-qld.csv", index=False)


# Stats

In [15]:
# should we search for synonyms?
#Litoria lorica
inattaxa[inattaxa['scientificName']=='Ranoidea lorica']


KeyError: 'scientificName'